In [1]:
!nvidia-smi
# 아무것도 안넣은 결과
# smart pad 결과
# 시형님 데이터 샘플링 코드 적용 결과


Sun Oct 23 01:29:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.56.06    Driver Version: 522.30       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   42C    P8    17W / 200W |      0MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import random
import os, sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import preprocessing
sys.path.append('../') # import를 위해 경로추가
from Modules import Utility as U
from Modules import CustomDataset 
from Modules import SmartPad
#torch.backends.cudnn.enabled = False

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=20)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--batch_size', type=int, default=20)
parser.add_argument('--seed', type=int, default=41)
parser.add_argument('--img_size', type=int, default=256)
parser.add_argument('--device', default='cuda')
parser.add_argument('--data_path', type=str, default='../data/')
parser.add_argument('--save_name', type=str, default="effnet_b4_300_smart_pad")

args = parser.parse_args('')
args

Namespace(batch_size=20, data_path='../data/', device='cuda', epochs=20, img_size=256, lr=0.0001, save_name='effnet_b4_300_smart_pad', seed=41)

In [4]:
U.seed_everything(args.seed)

In [5]:
df = pd.read_csv(args.data_path+'train_repaired.csv')

In [6]:
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [7]:
train_df, val_df, _, _ = U.train_test_split(df, df['artist'].values, test_size=0.2, random_state=args.seed)

In [8]:
train_img_paths, train_labels = U.get_data(train_df)
val_img_paths, val_labels = U.get_data(val_df)

In [9]:
# random flip
train_transform = A.Compose([
                            # SmartPad(),
                            A.Resize(args.img_size, args.img_size),
                            # A.RandomResizedCrop(height = args.img_size, 
                            #                     width = args.img_size,
                            #                     scale = (0.8,1)),
                            # A.HorizontalFlip(),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            # SmartPad(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [10]:
train_dataset = CustomDataset(args.data_path, train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(args.data_path, val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [11]:
class CustomModel(nn.Module):
    def __init__(self, number_of_classes :int):
        super(CustomModel, self).__init__()
        self.backborn = models.resnet101(pretrained=True)
        # for p in self.backborn.parameters():
        #     p.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.2),
            nn.Linear(512, number_of_classes)
        )
        
    def forward(self, x):
        x = self.backborn(x)
        x = self.classifier(x)
        return x

In [12]:
target_model = CustomModel(50)

In [13]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for item in tqdm(test_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = U.competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [14]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for item in tqdm(train_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            optimizer.zero_grad()

            model_pred = model(img)
            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model

In [15]:
model = target_model.to(args.device)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)

scheduler = None
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, args.device)

100%|██████████| 60/60 [00:20<00:00,  2.94it/s]


Epoch [1], Train Loss : [2.24008] Val Loss : [1.44964] Val F1 Score : [0.44144]


100%|██████████| 60/60 [00:20<00:00,  2.99it/s]


Epoch [2], Train Loss : [1.04976] Val Loss : [1.35924] Val F1 Score : [0.52604]


100%|██████████| 60/60 [00:20<00:00,  3.00it/s]


...

Epoch [18], Train Loss : [0.07214] Val Loss : [1.29832] Val F1 Score : [0.62198]


100%|██████████| 60/60 [00:20<00:00,  2.99it/s]


Epoch [19], Train Loss : [0.05132] Val Loss : [1.19107] Val F1 Score : [0.65441]


100%|██████████| 60/60 [00:20<00:00,  2.99it/s]

Epoch [20], Train Loss : [0.04813] Val Loss : [1.20602] Val F1 Score : [0.61129]


In [16]:
torch.save({
    'model': infer_model.state_dict(),
    'optim': optimizer.state_dict()
}, args.save_name + '.tar')

In [17]:
test_df = pd.read_csv(args.data_path + 'test.csv')

In [18]:
test_img_paths = U.get_data(test_df, infer=True)

In [19]:
test_dataset = CustomDataset(args.data_path, test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [20]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for item in tqdm(iter(test_loader)):
            img = item['image'].float().to(device)
            h = item['height']
            w = item['width']
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [21]:
preds = inference(infer_model, test_loader, args.device)

100%|██████████| 634/634 [01:48<00:00,  5.85it/s]

Done.


In [22]:
preds = le.inverse_transform(preds) 

In [23]:
submit = pd.read_csv(args.data_path + 'sample_submission.csv')

In [24]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Eugene Delacroix
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Vincent van Gogh


In [25]:
submit.to_csv('./' + args.save_name + '.csv', index=False)

In [26]:
torch.cuda.memory_reserved()

5284823040

In [27]:
torch.cuda.empty_cache()